In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from genparse import Float
from genparse.trace import TraceSWOR
from genparse.lm import MockLLM
from genparse.util import lark_guide, format_table
from genparse.proposal import TokenProposal, CharacterProposal
from arsenal.maths import assert_equal, random_dist

import asyncio
import nest_asyncio
nest_asyncio.apply()

WARNING 07-10 17:26:12 _custom_ops.py:14] Failed to import from vllm._C with ImportError('libcuda.so.1: cannot open shared object file: No such file or directory')


In [3]:
class enumerate_proposal_distribution:
    def __init__(self, proposal, prompt, context):
        tracer = TraceSWOR()
        P = Float.chart()
        while tracer.root.mass > 0:
            with tracer:
                (s, q, w) = asyncio.run(
                    proposal.sample_next_token(draw=tracer, prompt=prompt, context=context)
                )
                P[s] += q
        self.P = P
        self.tracer = tracer
    def _repr_html_(self):
        return format_table([[self.P, self.tracer.root.graphviz(fmt_edge=lambda i,a,j: a.replace(' ', '␣'))]])

In [4]:
if 1:
    V = {' a', ' b', ' ab', '▪'}
    
    grammar = r"""
        start: WS x EOS
        EOS: "▪"
        x: "a" | "b"
        WS: /[ ]/
    """

In [5]:
if 0:
    V = {' a', ' b', ' ac', '▪'}
    
    grammar = r"""
        start: WS x EOS
        EOS: "▪"
        x: "a" | "b"
        WS: /[ ]/
    """

In [6]:
llm = MockLLM(V=V, eos='▪', _p=None)
guide = lark_guide(grammar)

character_proposal = CharacterProposal(llm=llm, guide=guide)
token_proposal = TokenProposal(llm=llm, guide=guide, K=None)

prompt = ()
context = ()

# computes q(token) = Σ_S q(token, S)
chr = enumerate_proposal_distribution(character_proposal, prompt, context)
# P_tok = local product of experts
tok = enumerate_proposal_distribution(token_proposal, prompt, context)

#for v in V: print(repr(v), chr.P[v], tok.P[v])

display(chr.P.compare(tok.P))

,key,self,other,metric
0,b,0.333333,0.5,0.166667
1,a,0.666667,0.5,0.166667


In [7]:
chr

In [8]:
tok

key,value
b,0.5
a,0.5


In [14]:
from arsenal.iterextras import take

In [18]:
p_llm = llm.p_next(prompt + context)

# enumerate top K tokens
Ws = Float.chart(take(token_proposal.K, token_proposal.traverse_trie(context, p_llm)))

# compute distribution over wildcard tokens
P_wc = Float.chart({x: p for x, p in p_llm.items() if x not in Ws and p > 0})
P_wc

key,value
ab,0.25
▪,0.25


In [19]:
Ws

key,value
a,0.25
b,0.25
